In [1]:
import numpy as np
from scipy.spatial import distance_matrix

In [2]:
with open('../inputs/input19.txt', 'r') as f:
    strings = f.readlines()

scanners = []
for string in strings:
    if ',' in string:
        scanners[-1].append(np.array(list(map(int, string.strip().split(','))), dtype='int64'))
    elif '---' in string:
        scanners.append([])

In [3]:
scanner_sets = [set([tuple(point) for point in scan]) for scan in scanners]

In [4]:
np_scanners = [np.array(scanner) for scanner in scanners]

In [5]:
transformations = [
    lambda x: (x[0], x[1], x[2]),
    lambda x: (-x[0], x[1], x[2]),
    lambda x: (x[0], -x[1], x[2]),
    lambda x: (x[0], x[1], -x[2]),
    lambda x: (-x[0], -x[1], x[2]),
    lambda x: (x[0], -x[1], -x[2]),
    lambda x: (-x[0], x[1], -x[2]),
    lambda x: (-x[0], -x[1], -x[2]),
    lambda x: (x[1], x[0], x[2]),
    lambda x: (-x[1], x[0], x[2]),
    lambda x: (x[1], -x[0], x[2]),
    lambda x: (x[1], x[0], -x[2]),
    lambda x: (-x[1], -x[0], x[2]),
    lambda x: (x[1], -x[0], -x[2]),
    lambda x: (-x[1], x[0], -x[2]),
    lambda x: (-x[1], -x[0], -x[2]),
    lambda x: (x[2], x[1], x[0]),
    lambda x: (-x[2], x[1], x[0]),
    lambda x: (x[2], -x[1], x[0]),
    lambda x: (x[2], x[1], -x[0]),
    lambda x: (-x[2], -x[1], x[0]),
    lambda x: (x[2], -x[1], -x[0]),
    lambda x: (-x[2], x[1], -x[0]),
    lambda x: (-x[2], -x[1], -x[0]),
    lambda x: (x[0], x[2], x[1]),
    lambda x: (-x[0], x[2], x[1]),
    lambda x: (x[0], -x[2], x[1]),
    lambda x: (x[0], x[2], -x[1]),
    lambda x: (-x[0], -x[2], x[1]),
    lambda x: (x[0], -x[2], -x[1]),
    lambda x: (-x[0], x[2], -x[1]),
    lambda x: (-x[0], -x[2], -x[1]),
    lambda x: (x[1], x[2], x[0]),
    lambda x: (-x[1], x[2], x[0]),
    lambda x: (x[1], -x[2], x[0]),
    lambda x: (x[1], x[2], -x[0]),
    lambda x: (-x[1], -x[2], x[0]),
    lambda x: (x[1], -x[2], -x[0]),
    lambda x: (-x[1], x[2], -x[0]),
    lambda x: (-x[1], -x[2], -x[0]),
    lambda x: (x[2], x[0], x[1]),
    lambda x: (-x[2], x[0], x[1]),
    lambda x: (x[2], -x[0], x[1]),
    lambda x: (x[2], x[0], -x[1]),
    lambda x: (-x[2], -x[0], x[1]),
    lambda x: (x[2], -x[0], -x[1]),
    lambda x: (-x[2], x[0], -x[1]),
    lambda x: (-x[2], -x[0], -x[1]),
]

transformed_scanners = [[[transformations[i](x) for x in scan]for i in range(48)] for scan in scanners]

In [6]:
from itertools import product
from collections import defaultdict
from tqdm import tqdm
matched_scanners = set([0])
unmatched_scanners = set(range(1, len(scanners)))
matches = defaultdict(list)
rebased_scanners = [scanner_sets[i] if i == 0 else None for i in range(len(scanners))]
scanner_positions = [[0, 0, 0] if i == 0 else None for i in range(len(scanners))]

while len(unmatched_scanners) > 0 and len(matched_scanners) > 0:
    matched_this_round = []
    for matched_scanner in matched_scanners:
        for other_scanner in tqdm(unmatched_scanners):
            flag = False
            for point1, point2_i in product(rebased_scanners[matched_scanner], range(len(scanners[other_scanner]))):
                if point2_i > 14:
                    continue
                point1 = np.array(point1)
                for transform_i in range(0, 48):
                    transform = point1 - transformed_scanners[other_scanner][transform_i][point2_i]
                    transformed = [tuple(point + transform) for point in transformed_scanners[other_scanner][transform_i]]
                    if len([p for p in transformed if p in rebased_scanners[matched_scanner]]) >= 12:
                        flag = True
                        matches[matched_scanner].append(other_scanner)
                        matched_this_round.append(other_scanner)
                        rebased_scanners[other_scanner] = transformed.copy()
                        scanner_positions[other_scanner] = transform
                        break
                if flag:
                    break
        for matched in matched_this_round:
            unmatched_scanners.discard(matched)
    matched_scanners = matched_this_round.copy()

matches



100%|██████████| 1/1 [00:00<00:00, 10.99it/s]
0it [00:00, ?it/s]


defaultdict(list,
            {0: [5, 6, 23],
             5: [10, 13, 20],
             10: [12, 18, 26],
             13: [16],
             20: [4, 7],
             18: [21],
             26: [11],
             16: [17],
             4: [8, 24],
             7: [2],
             17: [9],
             8: [15, 19],
             24: [14, 29, 30],
             9: [27],
             15: [22],
             19: [3, 28],
             22: [1],
             3: [25]})

In [7]:
# part 1
len(set.union(*rebased_scanners))

378

In [8]:
# part 2
np.amax(distance_matrix(np.array(scanner_positions), np.array(scanner_positions), p=1))

13148.0